In [1]:
# 图像轮廓是具有相同颜色或者灰度的连续点的曲线,图片中物体的边缘
# 为了检测准确性,一般先对图片降噪,二值化 或者canny操作

In [1]:
import cv2
import numpy as np
cv2.__version__

'4.6.0'

In [3]:
# 注意 这个api返回有区别(大坑)
# findContours(image, mode, method[, contours[, hierarchy[, offset]]]) -> contours, hierarchy   4.6.0
# findContours(image, mode, method[, contours[, hierarchy[, offset]]]) -> image, contours, hierarch 3.4.1

# mode 查找轮廓的模式 返回值轮廓的索引编号方式
# 
# cv2.RETR_EXTERNAL =0 只检测外围轮廓
# cv2.RETR_LIST = 1 不建立等级关系,检测所有轮廓   不太好理解
# cv2.RETR_CCOMP =2 每层最多两级 不太好理解
# cv2.RETR_TREE =3 按照树形排列 从大到小,从右到左 人最好理解 最右边的最里面的轮廓为0

'''
通常情况下，我们会使用cv2.findContours() 函数来检测图像中的对象。
但是有时物体会在不同的位置。但在某些情况下，有一些形状会存在于其他形状里，就像是嵌套的图形。
在这种情况下，我们称外部形状为父类 parent，称内部的为子类 child；
这样，图像中的轮廓就有了某种联系。我们可以指定轮廓是如何相互连接的，
比如，这个轮廓是另一个轮廓的子类，或则是它的父类等等。这种关系的表示形式称为层级/层次结构。
'''
# method 保存轮廓的近似方法
# 
# cv2.CHAIN_APPROX_NONE 轮廓上的所有点
# cv2.CHAIN_APPROX_SIMPLE 只保存角点(比如四边形的四个顶点)
pass

In [7]:
img = cv2.imread('./ex_img.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

![img](./ex_img.png)

In [10]:
ex1 = cv2.Canny(img,100,200)

In [11]:
threshold ,ex2 =cv2.threshold(img,127,255,cv2.THRESH_BINARY)

In [13]:
show_ex = np.hstack((img,ex1,ex2))
cv2.imshow('ex',show_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.imwrite('show_ex.jpg',show_ex)

True

![img](./show_ex.jpg)

In [18]:
# 查找轮廓(注意版本api的返回值个数,类型)
contours, hierarchy = cv2.findContours(show_ex,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
print(type(contours),type(hierarchy))

<class 'tuple'> <class 'numpy.ndarray'>


In [19]:
len(contours),hierarchy.shape # hierarchy一般不关心

(1686, (1, 1686, 4))

In [25]:
# contours 里面是每一个元素是一个轮廓(折线边)所有点的坐标
contours[0],contours[5]

(array([[[326, 508]],
 
        [[423, 508]]], dtype=int32),
 array([[[310, 507]],
 
        [[309, 508]],
 
        [[268, 508]],
 
        [[322, 508]],
 
        [[311, 508]]], dtype=int32))

In [32]:
# 画轮廓 drawContours(image, contours, contourIdx, color
#                    [, thickness[, lineType[, hierarchy[, maxLevel[, offset]]]]]) -> image
draw_ex = show_ex.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,contours,-1,(0,0,255),thickness=1) # -1表示所有的轮廓

cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
# 想绘制指定的轮廓 ,发现数量特别多的时候,索引不好找

draw_ex = show_ex.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,contours,10,(0,0,255),thickness=1) # -1表示所有的轮廓

cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [54]:
# 利用轮廓的属性查找过滤

contours, hierarchy = cv2.findContours(show_ex,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# 计算轮廓的面积
Area = cv2.contourArea(contours[0])

# 计算轮廓的周长
arcLength = cv2.arcLength(contours[0],closed=True)
# print(Area,arcLength)

# 比如 找出10个面积最大的轮廓
Areas = np.array([cv2.contourArea(i) for i in contours])
index = np.argsort(Areas)
filter_contours =[]
for i in index[-10:]:
    filter_contours.append(contours[i])

# print(filter_contours)

# 画出筛选出来的轮廓
draw_ex = show_ex.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,filter_contours,-1,(0,0,255),thickness=1) # -1表示所有的轮廓

cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

0.0 194.0


In [55]:
# findContours找出来的轮廓可能比较复杂不平滑,可以使用多边形逼近的方法做适当的近似
# approxPolyDP(curve(轮廓), epsilon(阈值), closed[, approxCurve]) -> approxCurve

'''
（1）在曲线首尾两点A，B之间连接一条直线AB，该直线为曲线的弦；

（2）得到曲线上离该直线段距离最大的点C，计算其与AB的距离d；

（3）比较该距离与预先给定的阈值threshold的大小，如果小于threshold，则该直线段作为曲线的近似，该段曲线处理完毕。

（4）如果距离大于阈值，则用C将曲线分为两段AC和BC，并分别对两段取信进行1~3的处理。

（5）当所有曲线都处理完毕时，依次连接各个分割点形成的折线，即可以作为曲线的近似。

'''
pass


In [86]:
# 读取图片
img = cv2.imread('./ex_img.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 二值化
threshold ,ex2 =cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# 查找轮廓
contours, hierarchy = cv2.findContours(ex1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# print(len(contours))

# 画轮廓
draw_ex = img.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,contours,5,(0,0,255),thickness=2) # 找一个轮廓来演示

# 使用多边形逼近
newcontour =  cv2.approxPolyDP(contours[5],6,True) # 返回值就是一个轮廓
# 调用画轮廓的api
draw_ex = cv2.drawContours(draw_ex,[newcontour],0,(0,255,0),thickness=2) 
# 要符合格式要求,这里本身就是一个轮廓,封装成列表,然后索引0取出本身


cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [98]:
# 凸包 画出轮廓的外接凸多边形
# convexHull(points[, hull[, clockwise[, returnPoints]]]) -> hull

# 读取图片
img = cv2.imread('./ex_img.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
# 二值化
threshold ,ex2 =cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# 查找轮廓
contours, hierarchy = cv2.findContours(ex1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
# print(len(contours))

# 画轮廓
draw_ex = img.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,contours,1,(0,0,255),thickness=2) # 找一个轮廓来演示

# 使用多边形逼近
newcontour =  cv2.approxPolyDP(contours[1],3,True) # 返回值就是一个轮廓
# 调用画轮廓的api
draw_ex = cv2.drawContours(draw_ex,[newcontour],0,(0,255,0),thickness=2) 
# 要符合格式要求,这里本身就是一个轮廓,封装成列表,然后索引0取出本身

# 使用凸包
hall = cv2.convexHull(contours[1])
draw_ex = cv2.drawContours(draw_ex,[hall],0,(255,0,0),thickness=2) 

cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# 画外接矩形
# minAreaRect(points) -> retval 最小外接矩形 (旋转矩形) 调用 cv2.boxPoint()接口解析坐标点
# boundingRect(array) -> retval 最大外接矩形


### 旋转矩形在opencv中的表示
<img src="./20210831164909520.png" width=200 height=200>

In [111]:

img = cv2.imread('./2023-09-17_221452.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# 二值化
threshold ,ex2 =cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# 查找轮廓
contours, hierarchy = cv2.findContours(ex2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
# 画轮廓
draw_ex = img.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,contours,0,(0,0,255),thickness=1) # 找一个轮廓来演示

# 画最小外接矩形 
rect = cv2.minAreaRect(contours[0]) 
print(rect) # ((x,y),(w,h),θ)
# 解析4个点坐标
boxpoints =  cv2.boxPoints(rect)
boxpoints = boxpoints.astype(np.int64) # 注意用于画图坐标需要是整数(注意取整精度问题)
print(boxpoints)
draw_ex = cv2.drawContours(draw_ex,[boxpoints],0,(0,255,0),thickness=1)

# 画最大外接矩形
x,y,w,h = cv2.boundingRect(contours[0])
print(xywh)
cv2.rectangle(draw_ex,(x,y),(x+w,y+h),(255,0,0))

cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()

2
((144.19400024414062, 161.71783447265625), (141.90843200683594, 41.06708526611328), 74.87599182128906)
[[105  98]
 [145  87]
 [182 224]
 [142 235]]
(133, 88, 50, 138)


In [117]:
# 外接圆
# minEnclosingCircle(points) -> center, radius

img = cv2.imread('./2023-09-17_221452.png')
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

# 二值化
threshold ,ex2 =cv2.threshold(img,127,255,cv2.THRESH_BINARY)

# 查找轮廓
contours, hierarchy = cv2.findContours(ex2,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

# 画轮廓
draw_ex = img.copy()
draw_ex = cv2.cvtColor(draw_ex,cv2.COLOR_GRAY2BGR)
draw_ex = cv2.drawContours(draw_ex,contours,0,(0,0,255),thickness=1) # 找一个轮廓来演示

# 画最小外接圆
center, radius= cv2.minEnclosingCircle(contours[0])
center = [int(i) for i in center]
radius = int(radius)
cv2.circle(draw_ex ,center, radius,(255,0,0))

cv2.imshow('ex',draw_ex)
cv2.waitKey(0)
cv2.destroyAllWindows()